# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 79 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.01544


extracting tarball to tmp_2505.01544...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure images/WASP18b_emission_spectra_tp_V07_corr.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\linewidth,clip]{images/WASP18b_emission_spectra_tp_V07_corr.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure images/WASP18b_emission_retrievals_vmr_V06.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\linewidth,clip]{images/WASP18b_emission_r

Found 179 bibliographic references in tmp_2505.01544/aa50939-24.bbl.
Issues with the citations
syntax error in line 438: '=' expected
Retrieving document from  https://arxiv.org/e-print/2505.01606
extracting tarball to tmp_2505.01606...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.02127
extracting tarball to tmp_2505.02127...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.02434


extracting tarball to tmp_2505.02434...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.01544-b31b1b.svg)](https://arxiv.org/abs/2505.01544) | **Dark skies of the slightly eccentric WASP-18 b from its optical-to-infrared dayside emission**  |
|| A. Deline, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-05-06*|
|*Comments*| *31 pages, 17 figures, 13 tables (including the appendix); accepted for publication in A&A*|
|**Abstract**|            We analyse jointly phase-curve observations of the ultra-hot Jupiter WASP-18 b from the visible to the mid-infrared, using unpublished data from CHEOPS, TESS and Spitzer. We aim to characterise the planetary atmosphere with a consistent view over the large wavelength range covered using GCMs and retrieval analyses, and including JWST data. We obtain new ephemerides with unprecedented precisions of 1 second and 1.4 millisecond on the time of inferior conjunction and orbital period, respectively. We compute a planetary radius of $R_p = 1.1926 \pm 0.0077 R_J$ with a precision of 0.65%, or 550 km. Through a timing inconsistency with JWST, we discuss and confirm orbital eccentricity ($e = 0.00852 \pm 0.00091$), and constrain the argument of periastron to $\omega = 261.9^{+1.3}_{-1.4}$ deg. We show that the large dayside emission implies the presence of magnetic drag and super-solar metallicity. We find a steep thermally-inverted gradient in the planetary atmosphere, which is common for UHJs. We detect the presence of strong CO emission lines at 4.5 $\mu$m from an excess of dayside brightness in the Spitzer/IRAC/Ch2 passband. Using these models to constrain the reflected contribution in the CHEOPS passband, we derive an extremely low geometric albedo $A_g^\text{CHEOPS} = 0.027 \pm 0.011$.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.01606-b31b1b.svg)](https://arxiv.org/abs/2505.01606) | **X-ray Emission Signatures of Neutron Star Mergers**  |
|| C. Chen, <mark>Y. Wang</mark>, B. Zhang |
|*Appeared on*| *2025-05-06*|
|*Comments*| *18 pages, 14 figures. Submitted to ApJ*|
|**Abstract**|            Neutron star (NS) mergers, including both binary NS mergers and black hole - NS mergers, are multi-messenger sources detectable in both gravitational waves (GWs) and electromagnetic (EM) radiation. The expected EM emission signatures depend on the source's progenitor, merger remnant, and observer's line of sight (LoS). Widely discussed EM counterparts of NS mergers have been focused in the gamma-ray (in terms of short-duration gamma-ray bursts) and optical (in terms of kilonova) bands. In this paper, we show that X-ray emission carries unique post-merger signatures that are inaccessible in other EM bands and plays an important role in understanding the physics and geometry of these mergers. We consider several binary progenitor and central engine models and investigate X-ray emission signatures from the prompt phase immediately after the merger to the afterglow phase extending years later. For the prompt phase, we devise a general method for computing phenomenological X-ray light curves and spectra for structured jets viewed from any LoS, which can be applied to X-ray observations of NS mergers to constrain the geometry. The geometric constraints can in turn be used to model the afterglow and estimate a peak time and flux -- to preemptively determine afterglow characteristics would be monumental for follow-up observation campaigns of future GW sources. Finally, we provide constraints on the time window for X-ray counterpart searches of NS mergers across a range of luminosity distances and detector sensitivities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.02127-b31b1b.svg)](https://arxiv.org/abs/2505.02127) | **Evaluating the Accuracy of Non-parametric Galaxy Morphological Indicator Measurements in the CSST Imaging Survey**  |
|| Y. Luo, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-05-06*|
|*Comments*| *15 pages, 14 figures, to be published in PASP. Comments welcome*|
|**Abstract**|            The Chinese Space Station Telescope (CSST) is China's upcoming next-generation ultraviolet and optical survey telescope, with imaging resolution capabilities comparable to the Hubble Space Telescope (HST). In this study, we utilized a comprehensive sample of 3,679 CSST realistic mock galaxies constructed from HST CANDELS/GOODS-North deep imaging observations, with stellar masses $\log\left(M_{*} / M_{\odot}\right) > 9.0$ and redshifts $z < 2$. We evaluate the detection capabilities of CSST surveys and the accuracy in measuring the non-parametric morphological indicators ($C$, $A$, $Gini$, $M_{\rm 20}$, $A_{\rm O}$, $D_{\rm O}$) of galaxies. Our findings show that in terms of galaxy detection capabilities, CSST's deep field surveys can achieve the same level as HST's deep field observations; however, in wide-field surveys, CSST exhibits a significant deficiency in detecting high-redshift, low-mass, low-surface-brightness galaxies. Regarding the measurement of galaxy morphology, CSST's deep field surveys achieve high accuracy across all indicators except for the asymmetry indicator ($A$), whereas its wide-field surveys suffer from significant systematic biases. We thus provide simple correction functions to adjust the non-parametric morphological indicators obtained from CSST's wide-field and deep-field observations, thereby aligning CSST measurements with those from HST. This adjustment enables the direct application of non-parametric morphological classification methods originally developed for HST data to galaxies observed by CSST.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.02434-b31b1b.svg)](https://arxiv.org/abs/2505.02434) | **Early evolution of super-Eddington accretion flow in tidal disruption events**  |
|| E. Qiao, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-05-06*|
|*Comments*| *18 pages, 45 figures, accepted for publication by MNRAS*|
|**Abstract**|            Tidal disruption events (TDEs) are luminous black hole (BH) transient sources, which are detected mainly in X-ray and optical bands. It is generally believed that the X-ray emission in TDEs is produced by an accretion disc formed as the stellar debris accreted onto the central BH. The origin of the optical emission is not determined, but could be explained by the `reprocessing' model with the X-ray emission reprocessed into optical band by a surrounding optically thick envelope or outflow. In this paper, we performed radiation hydrodynamic simulations of super-Eddington accretion flow with Athena++ code in the environment of TDEs, i.e., injecting a continuous mass flow rate at the circularization radius in the form of $\dot M_{\rm inject} \propto t^{-5/3}$ for the mass supply rate. We show that a significant fraction of the matter in the accretion inflow are blowed off forming outflow, and the properties of the outflow are viewing-angle dependent. We further calculate the emergent spectra of such an inflow/outflow system for different viewing angles with the method of Monto Carlo radiative transfer. Based on the emergent spectra, we show that the observed features of TDEs, such as the X-ray and optical luminosities, the blackbody temperature of X-ray and optical emission and the corresponding emission radii, the ratio of X-ray luminosity to optical luminosity, as well as the evolution of these quantities can be explained in the framework of viewing-angle effect of super-Eddington accretion around a BH.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.01544/./images/geometric_albedo.png', 'tmp_2505.01544/./images/FpFs_plot.png', 'tmp_2505.01544/./images/W18b_JWST_e-w_ALL.png']
copying  tmp_2505.01544/./images/geometric_albedo.png to _build/html/
copying  tmp_2505.01544/./images/FpFs_plot.png to _build/html/
copying  tmp_2505.01544/./images/W18b_JWST_e-w_ALL.png to _build/html/
exported in  _build/html/2505.01544.md
    + _build/html/tmp_2505.01544/./images/geometric_albedo.png
    + _build/html/tmp_2505.01544/./images/FpFs_plot.png
    + _build/html/tmp_2505.01544/./images/W18b_JWST_e-w_ALL.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\host}{WASP-18}$
$\newcommand{\planet}{WASP-18 b}$
$\newcommand{\cheops}{CHEOPS}$
$\newcommand{\tess}{TESS}$
$\newcommand{\hst}{HST}$
$\newcommand{\spitzer}{Spitzer}$
$\newcommand{\gaia}{Gaia}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\kepler}{Kepler}$
$\newcommand{\batman}{\texttt{batman}}$
$\newcommand{\pipe}{PIPE}$
$\newcommand{\citepipe}{Brandeker et~al. (in~prep.)}$
$\newcommand{\pyratbay}{\textsc{Pyrat Bay}}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand{\missingref}{{\bf[REF TO BE ADDED]}}$
$\newcommand{\tbc}[1]{{\bf \color{orange} [TBC\expandafter\ifx\expandafter\relax\detokenize{#1}\relax\else\textnormal{\bf: #1}\fi]}}$
$\newcommand{\tbd}[1]{{\bf \color{orange} [TBD\expandafter\ifx\expandafter\relax\detokenize{#1}\relax\else\textnormal{\bf: #1}\fi]}}$
$\newcommand{\comment}[1]{{\bf \color{violet} [#1]}}$
$\newcommand{\au}{au}$
$\newcommand{\um}{\textmu m}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.25}$</div>



<div id="title">

# ${Dark skies of the slightly eccentric \planet \\from its optical-to-infrared dayside emission}$$\thanks{This work makes use of CHEOPS data from the Guaranteed Time Observation (GTO) programmes \texttt{CH\_PR100012} and \texttt{CH\_PR100016}.}$\textsuperscript{,}$\thanks{Raw and detrended light curves are available at the CDS via anonymous ftp to \href{ftp://cdsarc.u-strasbg.fr}{cdsarc.u-strasbg.fr} (\href{ftp://130.79.128.5}{130.79.128.5}) or via \url{http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.01544-b31b1b.svg)](https://arxiv.org/abs/2505.01544)<mark>Appeared on: 2025-05-06</mark> -  _31 pages, 17 figures, 13 tables (including the appendix); accepted for publication in A&A_

</div>
<div id="authors">

A. Deline, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** ${Ultra-hot Jupiters are gas giant exoplanets strongly irradiated by their star, which sets intense molecular dissociation leading to atmospheric chemistry}$ dominated by ions and atoms. These conditions inhibit ${day-to-night}$ heat redistribution resulting in ${large}$ temperature contrast. Phase-curve observations ${over several}$ passbands offer insights on the thermal structure and properties of these extreme atmospheres. We aim to perform a joint analysis of ${multiple}$ observations of $\planet$ from the visible to the mid-infrared, using unpublished data from $\cheops$ , $\tess$ and $\spitzer$ . Our purpose is to characterise the ${planetary atmosphere}$ with a consistent view over the large wavelength range covered, including $\jwst{data}$ . We implement a model for the planetary signal including transits, occultations, phase signal, ellipsoidal variations, Doppler boosting, and light-travel time. We fit jointly more than 250 ${{eclipse events} and derive atmospheric properties}$ using General Circulation Models and retrieval ${analyses}$ . ${We obtain}$ new ephemerides with unprecedented precisions of 1 second and 1.4 millisecond on the time of inferior conjunction and orbital period, respectively. We ${compute}$ a planetary radius of $R_p=1.1926\pm0.0077 R_J$ with a precision of 0.65 \% , or 550 km. Through a timing inconsistency with $\jwst$ , we ${discuss and confirm orbital eccentricity ($e=0.00852\pm0.00091$),}$ and constrain the argument of periastron to ${$\omega=261.9^{+1.3}_{-1.4} \deg$}$ . We show that the large dayside emission implies the presence of magnetic drag and super-solar metallicity. We ${find a steep thermally-inverted gradient in the planetary atmosphere, which is common for ultra-hot Jupiters}$ . ${We detect the presence of strong CO emission lines at 4.5 \textmu m from an excess of dayside brightness in the \spitzer /IRAC/Channel 2 passband. {Using these models to} constrain the reflected contribution in the \cheops  passband{, we derive an extremely low geometric albedo} $A_g^\text{\cheops }={0.027}\pm{0.011}$.}$ The orbital eccentricity remains a potential challenge for planetary dynamics that might require further study given the short-period massive planet and despite the young age of the system. The characterisation of the atmosphere of $\planet$ reveals the necessity to account for magnetic friction and super-solar metallicity to explain the full picture of the dayside emission. ${We find the planetary dayside to be extremely unreflective, but when juxtaposing \tess  and \cheops  data, we get hints of increased scattering efficiency in the visible, likely due to Rayleigh scattering.}$

</div>

<div id="div_fig1">

<img src="tmp_2505.01544/./images/geometric_albedo.png" alt="Fig7" width="100%"/>

**Figure 7. -** Estimated geometric albedo of $\planet$  based on the retrieval results in comparison to the passbands of $\cheops$  and $\tess$ . {The green line shows the geometric albedo with an \ch{H-} abundance derived by assuming chemical equilibrium {(extremely low $A_g$ values, reaching 0.03 at short wavelengths)}. For the purple line, this abundance is artificially reduced by 98\% {(yielding $A_g$ values comparable to our results in the $\cheops$  and $\tess$  passbands)}, while for the yellow case the \ch{H-} continuum is entirely removed as an opacity source. {We further explored the effect of removing} TiO and VO (red) as well as {Na and K} (blue).}{The impact of the former is negligible and the red curve lies hidden behind the yellow model. The latter reveals however more significant scattering spectral features, especially in the $\cheops$  and $\tess$  passbands.} The strong increase of $A_g$ towards lower wavelengths is caused by the $\lambda^{-4}$-dependence of Rayleigh scattering. (*fig:geometric_albedo*)

</div>
<div id="div_fig2">

<img src="tmp_2505.01544/./images/FpFs_plot.png" alt="Fig14" width="100%"/>

**Figure 14. -** Occultation depths as a function of wavelength. The black points with errorbars represent the measurement from this work ($\cheops$ , $\tess$  and $\spitzer$  -- 2 leftmost and 4 rightmost points) and from \cite{Coulombe2023} with $\jwst$(0.8--3 $\um$). The GCM simulations {including TiO and VO}(Section \ref{ssec:gcm}) with and without magnetic drag are shown in orange and light brown, respectively.
                        {The orange-filled diamonds mark the passband-integrated GCM values with magnetic drag ($\tau_\text{fric}=10^4 \text{s}$).}
                        The retrieval runs (Section \ref{ssec:retrieval}) are shown in {blue, pink, purple and green}(same colours as in Fig. \ref{fig:bright_temp} and Table \ref{table:retrievals}) depending on the data points included in the fit.
                        {An inset zoomed-in view of the $\cheops$ -to-1.75 $\um$  range is shown in the lower right corner for convenience.} (*fig:docc_wl*)

</div>
<div id="div_fig3">

<img src="tmp_2505.01544/./images/W18b_JWST_e-w_ALL.png" alt="Fig2" width="100%"/>

**Figure 2. -** Correlation plot of the posterior distribution of the eccentricity $e$ and argument of periastron $\omega$ of the orbit of $\planet$  that match the mid-occultation time $T_\text{occ}^\text{JWST}$ from \cite{Coulombe2023}. We used our posterior distribution on planetary parameters from Table \ref{tab:parameters}, and Gaussian priors on $e \cos\omega$ and $e \sin\omega${from \cite{Triaud2010_HJs} in blue, \cite{Nymeyer2011_WASP-18b} in orange, and \cite{Csizmadia2019_WASP-18b} in green. Overall, the 3 posterior distributions are all consistent with each other within $<1.2 \sigma$.}
                This plot made use of the package \texttt{corner.py}\citep{corner}. (*fig:e-w*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.01544"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

476  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

24  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
